# Interactive map

In [1]:
import os
import geopandas as gpd
import seaborn as sns

from h2ss import compare
from h2ss import data as rd

import pandas as pd
import numpy as np
import branca.colormap as cm
import folium
from folium.plugins import StripePattern, MeasureControl, MousePosition

In [2]:
ds, extent, exclusions = compare.load_all_data()

In [3]:
caverns = compare.capacity_function(
    ds=ds,
    extent=extent,
    exclusions=exclusions,
    cavern_diameter=85,
    cavern_height=120,
)

In [4]:
shape = rd.halite_shape(dat_xr=ds).buffer(1000).buffer(-1000)

In [5]:
caverns_plt = caverns[
    [
        "geometry",
        "cavern_depth",
        "capacity",
        "Bathymetry",
        "NetToGross",
        "ThicknessNet",
    ]
].copy()
caverns_plt["capacity_str"] = [f"{x:.2f}" for x in caverns["capacity"]]
caverns_plt["Bathymetry"] = [f"{-x:.2f}" for x in caverns["Bathymetry"]]
caverns_plt["NetToGross"] = [f"{x * 100:.0f}" for x in caverns["NetToGross"]]
caverns_plt["ThicknessNet"] = [f"{x:,.2f}" for x in caverns["ThicknessNet"]]
caverns_plt["cavern_depth"] = [f"{x:,.2f}" for x in caverns["cavern_depth"]]
caverns_plt["geometry"] = caverns_plt["geometry"].buffer(100)

In [6]:
exclusions["wind_farms"]["cap"] = [
    f"{x:,.0f}" for x in exclusions["wind_farms"]["cap"]
]

In [7]:
exclusions["shipwrecks"]["VESSELNAME"] = exclusions["shipwrecks"][
    "VESSELNAME"
].fillna("None")

In [8]:
# create exclusion buffer
buffer = (
    pd.concat(
        [
            exclusions["wells_b"],
            exclusions["shipwrecks_b"],
            exclusions["shipping_b"],
            exclusions["cables_b"],
        ]
    )
    .dissolve()
    .overlay(gpd.GeoDataFrame(geometry=shape))
)

In [9]:
minx, miny, maxx, maxy = shape.to_crs(4326).bounds.values[0]
m = folium.Map(
    tiles="cartodbvoyager",
    control_scale=True,
    location=[np.mean((miny, maxy)), np.mean((minx, maxx))],
    zoom_start=11,
    min_lat=miny,
    max_lat=maxy,
    min_lon=minx,
    max_lon=maxx,
)

folium.GeoJson(
    shape.to_crs(4326),
    name="Kish Basin boundary",
    style_function=lambda feature: {
        "fillColor": "none",
        "color": "darkslategrey",
        "weight": 1,
    },
    tooltip="Kish Basin boundary",
).add_to(m)

folium.GeoJson(
    buffer.to_crs(4326),
    name="Exclusion buffer",
    style_function=lambda feature: {
        "fillPattern": StripePattern(
            angle=45,
            color="slategrey",
            space_color="none",
        ),
        "color": "slategrey",
        "weight": 0.5,
        "dashArray": "5, 5",
    },
    tooltip="Exclusion buffer",
).add_to(m)

folium.GeoJson(
    exclusions["wind_farms"].to_crs(4326),
    name="Wind farms",
    tooltip=folium.GeoJsonTooltip(
        fields=["name", "cap"], aliases=["Wind farm", "Capacity [MW]"]
    ),
    style_function=lambda feature: {
        "fillColor": "none",
        "color": "royalblue",
        "fillPattern": StripePattern(
            angle=135, color="royalblue", space_color="none"
        ),
        "weight": 1,
    },
).add_to(m)

folium.GeoJson(
    exclusions["shipwrecks"].to_crs(4326),
    name="Shipwrecks",
    tooltip=folium.GeoJsonTooltip(
        fields=["VESSELNAME", "VESSELTYPE"], aliases=["Vessel", "Type"]
    ),
    marker=folium.Marker(icon=folium.Icon(icon="sailboat", prefix="fa")),
    style_function=lambda feature: {"markerColor": "gray"},
).add_to(m)
folium.GeoJson(
    exclusions["wells"].to_crs(4326),
    name="Exploration wells",
    marker=folium.Marker(icon=folium.Icon(icon="oil-well", prefix="fa")),
    style_function=lambda feature: {"markerColor": "gray"},
    tooltip=folium.GeoJsonTooltip(
        fields=["RIG_NAME", "OPERATOR"], aliases=["Rig", "Operator"]
    ),
).add_to(m)

folium.GeoJson(
    exclusions["shipping"].to_crs(4326),
    name="Frequent shipping routes",
    color="crimson",
    tooltip="Frequent shipping route",
).add_to(m)
folium.GeoJson(
    exclusions["cables"].to_crs(4326),
    name="Subsea cables",
    color="royalblue",
    tooltip="Subsea cable",
).add_to(m)

colormap = cm.StepColormap(
    list(sns.color_palette("rocket_r")),
    vmin=caverns_plt.capacity.min(),
    vmax=caverns_plt.capacity.max(),
)
caverns_dict = caverns_plt.reset_index().set_index("index")["capacity"]
color_dict = {key: colormap(caverns_dict[key]) for key in caverns_dict.keys()}
folium.GeoJson(
    caverns_plt.to_crs(4326),
    style_function=lambda feature: {
        "fillColor": color_dict[int(feature["id"])],
        "color": "darkslategrey",
        "weight": 0.5,
        "fillOpacity": 1,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=[
            "capacity_str",
            "cavern_depth",
            "Bathymetry",
            "ThicknessNet",
            "NetToGross",
        ],
        aliases=[
            "Cavern capacity [GWh]",
            "Cavern top depth [m]",
            "Water depth [m]",
            "Net halite thickness [m]",
            "Net-to-gross ratio [%]",
        ],
    ),
    name="Caverns",
    smooth_factor=0,
).add_to(m)
colormap.caption = "Cavern capacity [GWh]"
colormap.add_to(m)

folium.LayerControl().add_to(m)
MeasureControl().add_to(m)
MousePosition().add_to(m)

m